In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

from iso3166 import countries

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

import pycountry

sns.set_context("talk")
plt.style.use('ggplot')
mpl.rcParams['axes.labelpad']='20'

### data en https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HTTWYL

In [2]:
df = pd.read_csv("../datos/protests.tab", sep='\t')
df = df[df["protest"] ==1.0]
df["ccode"] = df["ccode"].astype("int32")
display(df.head(2))
print(df.shape)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001.0,Canada,20,1990,3.0,1,1.0,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. Great Canadian train journeys into history;...,Canada s railway passenger system was finally...
1,201990002.0,Canada,20,1990,3.0,1,2.0,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. Autonomy s Cry Revived in Quebec The New Yo...,protestors were only identified as young peop...


(13770, 31)


In [7]:
paises = {}
for i, protest in df.iterrows():
    pais = protest["country"]
    año = protest["year"]

    if pais in paises:
        if año in paises[pais]["cuenta"]:
            paises[pais]["cuenta"][año] += 1
        else:
            paises[pais]["cuenta"][año]=1
    else:
        try:
            paises[pais]={
                "codigo":pycountry.countries.search_fuzzy(pais)[0].alpha_3,
                "cuenta":{
                año:1
                }
                
            }
        except:
            pass



In [8]:
df_cuenta = pd.concat({(k, v["codigo"]): pd.Series(v["cuenta"]) for k,v in paises.items()}, axis=0)
df_unstacked= df_cuenta.unstack()
df_unstacked = df_unstacked.reset_index()
df_unstacked = df_unstacked.rename(columns={"level_0": "Country Name", "level_1":"Country Code"})
df_unstacked = df_unstacked.fillna(0)
df_unstacked

,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,AFG,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,4.0,1.0,1.0,6.0,5.0,4.0,3.0,2.0
1,Albania,ALB,5.0,8.0,3.0,2.0,1.0,0.0,1.0,11.0,...,0.0,6.0,3.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0
2,Algeria,DZA,2.0,2.0,6.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,2.0,11.0,3.0,0.0,2.0,0.0
3,Angola,AGO,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2.0,0.0,3.0,1.0,1.0,0.0
4,Argentina,ARG,8.0,2.0,2.0,3.0,7.0,3.0,6.0,5.0,...,1.0,3.0,0.0,2.0,2.0,0.0,5.0,1.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Venezuela,VEN,9.0,12.0,26.0,8.0,11.0,6.0,13.0,10.0,...,16.0,7.0,2.0,0.0,7.0,16.0,8.0,10.0,22.0,0.0
147,Vietnam,VNM,0.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
148,Yemen,YEM,3.0,0.0,2.0,0.0,3.0,2.0,1.0,1.0,...,14.0,6.0,58.0,4.0,3.0,33.0,11.0,1.0,0.0,0.0
149,Zambia,ZMB,1.0,1.0,5.0,1.0,4.0,3.0,5.0,12.0,...,2.0,2.0,6.0,6.0,4.0,9.0,0.0,6.0,1.0,1.0


In [13]:
years = list(range(1990,2018))

In [15]:
for year in years:
    data=dict(
        type = 'choropleth',
        locations = df_unstacked['Country Code'],
        z = df_unstacked[year],
        text = df_unstacked['Country Name'],
        colorscale = 'PuRd',
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = '#',
        zauto=False,
        zmax=20,


    )

    layout = dict(geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular',

        ))

    fig = go.Figure(data = [data], layout = layout)

    fig.update_layout(
        title="# de protestas en el " + str(year),
        font=dict(
            family="Courier New, monospace",
            size=28,
            color="#7f7f7f"
        )

    )
    # plot(fig,filename='../outputs/diabetes.html')
    iplot(fig)

In [2]:
8.3 + 91.7

100.0